# Redes Neuronales Profundas para Deteccion de Desinformacion

Implementacion de redes neuronales profundas (DNN) para deteccion de noticias falsas usando el dataset Truth_Seeker_Model_Dataset.

Utilizo arquitecturas avanzadas con capas densas, LSTM, CNN y tecnicas de regularizacion.

## Objetivos
1. Implementar arquitecturas de redes neuronales profundas
2. Utilizar embeddings y capas especializadas para texto
3. Aplicar tecnicas de regularizacion para evitar overfitting
4. Optimizar hiperparametros para maximizar rendimiento
5. Comparar diferentes arquitecturas DNN
6. Evaluar contra modelos tradicionales

Notebook optimizado para Google Colab con GPU.

In [1]:
# Verificar entorno y configurar dependencias
import sys
import subprocess

# Detectar si estamos en Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Ejecutando en Google Colab")
    # Instalar dependencias para Colab
    !pip install tensorflow plotly scikit-learn
    # Montar Google Drive si es necesario
    from google.colab import drive
    # drive.mount('/content/drive')
else:
    print("Ejecutando en entorno local")
    # Verificar dependencias locales
    packages = ['tensorflow', 'plotly', 'scikit-learn']
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print(f"Entorno: {'Google Colab' if IN_COLAB else 'Local'}")

Ejecutando en Google Colab
Entorno: Google Colab


In [2]:
# Importacion de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, GRU, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Input, Concatenate, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Procesamiento de texto
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Machine learning
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

# Reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

# Configuracion
if IN_COLAB:
    plt.style.use('default')
else:
    plt.rcParams['figure.figsize'] = (10, 6)

# Verificar GPU
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"Eager execution: {tf.executing_eagerly()}")

# NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("Librerias importadas correctamente")

TensorFlow version: 2.19.0
GPU disponible: False
Eager execution: True
Librerias importadas correctamente


## Carga de Datos Truth Seeker

Cargo el dataset Truth_Seeker_Model_Dataset para entrenar las redes neuronales profundas.

In [3]:
# Cargar datasets Truth Seeker
if IN_COLAB:
    # Para Colab - ajustar segun donde subas los datos
    df_features = pd.read_csv('Features_For_Traditional_ML_Techniques.csv')
    df_truth_seeker = pd.read_csv('Truth_Seeker_Model_Dataset.csv')
else:
    # Para ejecucion local
    df_features = pd.read_csv('../dataset1/Features_For_Traditional_ML_Techniques.csv')
    df_truth_seeker = pd.read_csv('../dataset1/Truth_Seeker_Model_Dataset.csv')

print(f"Features dataset: {df_features.shape[0]} filas, {df_features.shape[1]} columnas")
print(f"Truth Seeker dataset: {df_truth_seeker.shape[0]} filas, {df_truth_seeker.shape[1]} columnas")

# Usar Truth Seeker dataset que contiene el texto
df = df_truth_seeker.copy()

print("\nPrimeras filas dataset:")
print(df.head())

# Verificar columnas disponibles
text_columns = [col for col in df.columns if 'statement' in col.lower() or 'tweet' in col.lower()]
target_columns = [col for col in df.columns if 'target' in col.lower()]

print(f"\nColumnas de texto: {text_columns}")
print(f"Columnas target: {target_columns}")

# Definir columnas a usar
text_col = 'statement' if 'statement' in df.columns else text_columns[0] if text_columns else None
target_col = 'BinaryNumTarget' if 'BinaryNumTarget' in df.columns else 'majority_target' if 'majority_target' in df.columns else target_columns[0] if target_columns else None

print(f"\nUsando columna texto: {text_col}")
print(f"Usando columna target: {target_col}")

if target_col:
    print(f"\nDistribucion target:")
    print(df[target_col].value_counts())

Features dataset: 134198 filas, 64 columnas
Truth Seeker dataset: 134198 filas, 9 columnas

Primeras filas dataset:
   Unnamed: 0      author                                          statement  \
0           0  D.L. Davis  End of eviction moratorium means millions of A...   
1           1  D.L. Davis  End of eviction moratorium means millions of A...   
2           2  D.L. Davis  End of eviction moratorium means millions of A...   
3           3  D.L. Davis  End of eviction moratorium means millions of A...   
4           4  D.L. Davis  End of eviction moratorium means millions of A...   

   target  BinaryNumTarget                 manual_keywords  \
0    True              1.0  Americans, eviction moratorium   
1    True              1.0  Americans, eviction moratorium   
2    True              1.0  Americans, eviction moratorium   
3    True              1.0  Americans, eviction moratorium   
4    True              1.0  Americans, eviction moratorium   

                              

In [4]:
# Limpieza y preparacion inicial
if text_col and target_col:
    print(f"Dataset inicial: {len(df)} filas")
    print(f"Declaraciones unicas: {df[text_col].nunique()}")

    # Eliminar nulos
    df_clean = df.dropna(subset=[text_col, target_col]).copy()
    print(f"Despues de eliminar nulos: {len(df_clean)} filas")

    # Eliminar duplicados
    duplicates = df_clean.duplicated(subset=[text_col], keep=False)
    if duplicates.sum() > 0:
        print(f"Duplicados encontrados: {duplicates.sum()}")
        df_clean = df_clean.drop_duplicates(subset=[text_col], keep='first')
        print(f"Despues de eliminar duplicados: {len(df_clean)} filas")

    # Eliminar majority_target si existe para evitar data leakage
    if 'majority_target' in df_clean.columns and target_col != 'majority_target':
        df_clean = df_clean.drop(columns=['majority_target'])
        print("Eliminada columna majority_target para evitar data leakage")

    # Distribucion final
    print(f"\nDistribucion final:")
    print(df_clean[target_col].value_counts())

    # Visualizar distribucion
    fig = px.pie(values=df_clean[target_col].value_counts().values,
                 names=df_clean[target_col].value_counts().index,
                 title=f"Distribucion {target_col}")
    fig.show()

else:
    print("Error: No se encontraron columnas de texto o target")
    df_clean = df.copy()

Dataset inicial: 134198 filas
Declaraciones unicas: 1058
Despues de eliminar nulos: 134198 filas
Duplicados encontrados: 134167
Despues de eliminar duplicados: 1058 filas

Distribucion final:
BinaryNumTarget
1.0    579
0.0    479
Name: count, dtype: int64


## Preprocesamiento de Texto para DNN

Preparo el texto especificamente para redes neuronales profundas con tokenizacion y secuencias.

In [5]:
# Funcion de preprocesamiento para DNN
def preprocess_text_for_dnn(text):
    """Preprocesa texto para redes neuronales"""
    if pd.isna(text):
        return ""

    # Convertir a string y minusculas
    text = str(text).lower()

    # Eliminar caracteres especiales pero mantener espacios
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Eliminar espacios multiples
    text = re.sub(r'\s+', ' ', text)

    # Eliminar palabras muy cortas
    words = text.split()
    words = [word for word in words if len(word) >= 2]

    return ' '.join(words)

if text_col:
    print("Preprocesando texto para DNN...")
    df_clean['statement_processed'] = df_clean[text_col].apply(preprocess_text_for_dnn)

    # Eliminar textos vacios
    df_clean = df_clean[df_clean['statement_processed'].str.len() > 0].copy()
    print(f"Filas despues de eliminar textos vacios: {len(df_clean)}")

    # Analizar longitudes
    df_clean['text_length'] = df_clean['statement_processed'].str.len()
    df_clean['word_count'] = df_clean['statement_processed'].str.split().str.len()

    print("\nEstadisticas longitud texto:")
    print(df_clean[['text_length', 'word_count']].describe())

    # Ejemplos preprocesamiento
    print("\nEjemplos preprocesamiento:")
    for i in range(min(3, len(df_clean))):
        original = str(df_clean[text_col].iloc[i])[:100]
        processed = df_clean['statement_processed'].iloc[i][:100]
        print(f"Original: {original}...")
        print(f"Procesado: {processed}...")
        print()

Preprocesando texto para DNN...
Filas despues de eliminar textos vacios: 1058

Estadisticas longitud texto:
       text_length   word_count
count  1058.000000  1058.000000
mean     89.819471    15.201323
std      38.414713     6.554200
min      20.000000     3.000000
25%      62.000000    11.000000
50%      82.000000    14.000000
75%     111.000000    19.000000
max     287.000000    46.000000

Ejemplos preprocesamiento:
Original: End of eviction moratorium means millions of Americans could lose their housing in the middle of a p...
Procesado: end of eviction moratorium means millions of americans could lose their housing in the middle of pan...

Original: The Trump administration worked to free 5,000 Taliban prisoners....
Procesado: the trump administration worked to free 5000 taliban prisoners...

Original: In Afghanistan, over 100 billion dollars spent on military contracts....
Procesado: in afghanistan over 100 billion dollars spent on military contracts...



In [6]:
# Analizar distribucion de longitudes
if 'word_count' in df_clean.columns:
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=['Longitud Caracteres', 'Cantidad Palabras'])

    fig.add_trace(go.Histogram(x=df_clean['text_length'], nbinsx=50, name='Chars'), row=1, col=1)
    fig.add_trace(go.Histogram(x=df_clean['word_count'], nbinsx=50, name='Words'), row=1, col=2)

    fig.update_layout(title="Analisis Longitudes Texto", height=400, showlegend=False)
    fig.show()

    # Percentiles para parametros de red
    word_percentiles = np.percentile(df_clean['word_count'], [50, 75, 90, 95, 99])
    print(f"\nPercentiles cantidad palabras:")
    for p, val in zip([50, 75, 90, 95, 99], word_percentiles):
        print(f"  {p}%: {val:.0f} palabras")

    # Parametros para DNN
    MAX_SEQUENCE_LENGTH = int(np.percentile(df_clean['word_count'], 95))
    MAX_VOCAB_SIZE = 20000

    print(f"\nParametros DNN:")
    print(f"  Longitud maxima secuencia: {MAX_SEQUENCE_LENGTH}")
    print(f"  Tamano maximo vocabulario: {MAX_VOCAB_SIZE}")
else:
    print("Error: No se pudo procesar el texto")
    MAX_SEQUENCE_LENGTH = 100
    MAX_VOCAB_SIZE = 10000


Percentiles cantidad palabras:
  50%: 14 palabras
  75%: 19 palabras
  90%: 24 palabras
  95%: 28 palabras
  99%: 37 palabras

Parametros DNN:
  Longitud maxima secuencia: 28
  Tamano maximo vocabulario: 20000


## Tokenizacion y Preparacion Secuencias

Convierto texto en secuencias numericas para las redes neuronales.

In [7]:
# Preparar datos para division
if 'statement_processed' in df_clean.columns and target_col in df_clean.columns:
    X = df_clean['statement_processed'].values
    y = df_clean[target_col].values

    # Codificar etiquetas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    print(f"Etiquetas: {label_encoder.classes_} -> {np.unique(y_encoded)}")

    # Division train/test ANTES de tokenizar
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    print(f"\nEntrenamiento: {len(X_train)} muestras")
    print(f"Prueba: {len(X_test)} muestras")
    print(f"Distribucion entrenamiento: {np.bincount(y_train)}")
    print(f"Distribucion prueba: {np.bincount(y_test)}")

    # Tokenizacion SOLO con datos entrenamiento
    print("\nTokenizando textos...")
    tokenizer = Tokenizer(
        num_words=MAX_VOCAB_SIZE,
        oov_token="<OOV>",
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    )

    tokenizer.fit_on_texts(X_train)

    # Convertir a secuencias
    X_train_sequences = tokenizer.texts_to_sequences(X_train)
    X_test_sequences = tokenizer.texts_to_sequences(X_test)

    # Aplicar padding
    X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
    X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

    print(f"\nForma entrenamiento: {X_train_padded.shape}")
    print(f"Forma prueba: {X_test_padded.shape}")
    print(f"Vocabulario total: {len(tokenizer.word_index)}")
    print(f"Vocabulario efectivo: {min(len(tokenizer.word_index), MAX_VOCAB_SIZE)}")

else:
    print("Error: Datos no disponibles para tokenizacion")

Etiquetas: [0. 1.] -> [0 1]

Entrenamiento: 846 muestras
Prueba: 212 muestras
Distribucion entrenamiento: [383 463]
Distribucion prueba: [ 96 116]

Tokenizando textos...

Forma entrenamiento: (846, 28)
Forma prueba: (212, 28)
Vocabulario total: 3406
Vocabulario efectivo: 3406


In [8]:
# Analizar longitudes despues padding
if 'X_train_padded' in locals():
    train_lengths = [len([token for token in seq if token != 0]) for seq in X_train_padded]
    test_lengths = [len([token for token in seq if token != 0]) for seq in X_test_padded]

    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=['Longitudes Entrenamiento', 'Longitudes Prueba'])

    fig.add_trace(go.Histogram(x=train_lengths, nbinsx=50, name='Train'), row=1, col=1)
    fig.add_trace(go.Histogram(x=test_lengths, nbinsx=50, name='Test'), row=1, col=2)

    fig.update_layout(title="Distribucion Longitudes Reales (sin padding)", height=400, showlegend=False)
    fig.show()

    print(f"Longitud promedio entrenamiento: {np.mean(train_lengths):.1f}")
    print(f"Longitud promedio prueba: {np.mean(test_lengths):.1f}")
    print(f"Secuencias truncadas: {(np.array(train_lengths) == MAX_SEQUENCE_LENGTH).mean() * 100:.1f}%")

    # Ejemplos tokenizacion
    print("\nEjemplos tokenizacion:")
    for i in range(min(2, len(X_train))):
        print(f"\nEjemplo {i+1}:")
        print(f"Texto: {X_train[i][:100]}...")
        print(f"Secuencia: {X_train_sequences[i][:20]}...")
        print(f"Con padding: {X_train_padded[i][:20]}...")
        print(f"Longitudes - Original: {len(X_train_sequences[i])}, Con padding: {len(X_train_padded[i])}")
else:
    print("Datos no disponibles para analisis")

Longitud promedio entrenamiento: 15.0
Longitud promedio prueba: 14.9
Secuencias truncadas: 5.6%

Ejemplos tokenizacion:

Ejemplo 1:
Texto: the federal government owns tens of thousands of properties that are vacant or underused...
Secuencia: [2, 89, 144, 814, 815, 4, 253, 4, 1363, 11, 10, 1364, 65, 1365]...
Con padding: [   2   89  144  814  815    4  253    4 1363   11   10 1364   65 1365
    0    0    0    0    0    0]...
Longitudes - Original: 14, Con padding: 28

Ejemplo 2:
Texto: year ago we had the lowest unemployment in history without using taxpayers money...
Secuencia: [53, 193, 28, 90, 2, 145, 291, 3, 129, 162, 816, 292, 293]...
Con padding: [ 53 193  28  90   2 145 291   3 129 162 816 292 293   0   0   0   0   0
   0   0]...
Longitudes - Original: 13, Con padding: 28


## Arquitecturas Redes Neuronales Profundas

Implemento diferentes arquitecturas DNN para clasificacion de texto.

In [9]:
# Funciones para crear modelos DNN

def create_basic_dnn(vocab_size, embedding_dim=128, max_length=MAX_SEQUENCE_LENGTH):
    """Red neuronal profunda basica con embeddings"""
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        GlobalMaxPooling1D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

def create_lstm_model(vocab_size, embedding_dim=128, max_length=MAX_SEQUENCE_LENGTH):
    """Modelo LSTM para procesamiento secuencial"""
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True),
        LSTM(64, dropout=0.3, recurrent_dropout=0.3),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

def create_gru_model(vocab_size, embedding_dim=128, max_length=MAX_SEQUENCE_LENGTH):
    """Modelo GRU como alternativa a LSTM"""
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        GRU(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True),
        GRU(64, dropout=0.3, recurrent_dropout=0.3),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

def create_cnn_model(vocab_size, embedding_dim=128, max_length=MAX_SEQUENCE_LENGTH):
    """Modelo CNN para capturar patrones locales"""
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Conv1D(128, 5, activation='relu'),
        MaxPooling1D(2),
        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),
        Conv1D(32, 3, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

print("Funciones arquitecturas DNN definidas")

Funciones arquitecturas DNN definidas


In [10]:
# Modelos avanzados

def create_hybrid_model(vocab_size, embedding_dim=128, max_length=MAX_SEQUENCE_LENGTH):
    """Modelo hibrido CNN + LSTM"""
    inputs = Input(shape=(max_length,))

    # Embedding compartido
    embedding = Embedding(vocab_size, embedding_dim)(inputs)

    # Rama CNN
    cnn_branch = Conv1D(64, 3, activation='relu')(embedding)
    cnn_branch = MaxPooling1D(2)(cnn_branch)
    cnn_branch = Conv1D(32, 3, activation='relu')(cnn_branch)
    cnn_branch = GlobalMaxPooling1D()(cnn_branch)

    # Rama LSTM
    lstm_branch = LSTM(64, dropout=0.3, recurrent_dropout=0.3)(embedding)

    # Combinar
    combined = Concatenate()([cnn_branch, lstm_branch])

    # Capas densas
    dense = Dense(256, activation='relu')(combined)
    dense = Dropout(0.5)(dense)
    dense = Dense(128, activation='relu')(dense)
    dense = Dropout(0.3)(dense)
    outputs = Dense(1, activation='sigmoid')(dense)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

def create_advanced_dnn(vocab_size, embedding_dim=128, max_length=MAX_SEQUENCE_LENGTH):
    """DNN muy profundo con batch normalization"""
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        GlobalMaxPooling1D(),

        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),

        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),

        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),

        Dense(64, activation='relu'),
        Dropout(0.1),

        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

print("Funciones modelos avanzados definidas")

Funciones modelos avanzados definidas


## Entrenamiento de Modelos

Entreno diferentes arquitecturas y comparo rendimiento.

In [11]:
# Configuracion entrenamiento
if 'tokenizer' in locals():
    VOCAB_SIZE = min(len(tokenizer.word_index) + 1, MAX_VOCAB_SIZE)
    EMBEDDING_DIM = 128
    EPOCHS = 20
    BATCH_SIZE = 32
    VALIDATION_SPLIT = 0.2

    print(f"Configuracion entrenamiento:")
    print(f"  Vocabulario: {VOCAB_SIZE}")
    print(f"  Embedding dim: {EMBEDDING_DIM}")
    print(f"  Epocas: {EPOCHS}")
    print(f"  Batch size: {BATCH_SIZE}")
    print(f"  Validacion: {VALIDATION_SPLIT}")

    # Callbacks
    def get_callbacks(model_name):
        return [
            EarlyStopping(
                monitor='val_accuracy',
                patience=5,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=3,
                min_lr=1e-7,
                verbose=1
            )
        ]

    print("Configuracion lista para entrenamiento")
else:
    print("Error: Tokenizer no disponible")

Configuracion entrenamiento:
  Vocabulario: 3407
  Embedding dim: 128
  Epocas: 20
  Batch size: 32
  Validacion: 0.2
Configuracion lista para entrenamiento


In [12]:
# Funcion entrenar y evaluar
def train_and_evaluate_model(model_func, model_name, verbose=1):
    """Entrena y evalua modelo"""
    print(f"\n{'='*60}")
    print(f"Entrenando: {model_name}")
    print(f"{'='*60}")

    # Crear modelo
    model = model_func(VOCAB_SIZE, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH)

    if verbose:
        print(f"\nArquitectura {model_name}:")
        model.summary()

    # Entrenar
    history = model.fit(
        X_train_padded, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=VALIDATION_SPLIT,
        callbacks=get_callbacks(model_name),
        verbose=1 if verbose else 0
    )

    # Evaluar
    test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=0)

    # Predicciones
    y_pred_proba = model.predict(X_test_padded, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()

    # Metricas
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)

    print(f"\nResultados {model_name}:")
    print(f"  Loss: {test_loss:.4f}")
    print(f"  Accuracy: {test_accuracy:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  ROC-AUC: {auc:.4f}")

    return {
        'model': model,
        'model_name': model_name,
        'history': history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'f1_score': f1,
        'roc_auc': auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

print("Funcion entrenamiento definida")

Funcion entrenamiento definida


In [13]:
# Entrenar modelos principales
if 'X_train_padded' in locals() and 'y_train' in locals():
    models_to_train = [
        (create_basic_dnn, "DNN Basico"),
        (create_cnn_model, "CNN 1D"),
        (create_lstm_model, "LSTM"),
        (create_gru_model, "GRU")
    ]

    results = []

    # Entrenar cada modelo
    for i, (model_func, model_name) in enumerate(models_to_train):
        try:
            verbose = (i == 0)  # Solo mostrar detalles del primer modelo
            result = train_and_evaluate_model(model_func, model_name, verbose=verbose)
            results.append(result)

        except Exception as e:
            print(f"Error entrenando {model_name}: {str(e)}")
            continue

    print(f"\nModelos principales entrenados: {len(results)}")
else:
    print("Datos no disponibles para entrenamiento")
    results = []


Entrenando: DNN Basico

Arquitectura DNN Basico:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.5555 - loss: 0.6906 - val_accuracy: 0.5294 - val_loss: 0.6899 - learning_rate: 0.0010
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5549 - loss: 0.6825 - val_accuracy: 0.5294 - val_loss: 0.6844 - learning_rate: 0.0010
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5652 - loss: 0.6602 - val_accuracy: 0.5294 - val_loss: 0.6596 - learning_rate: 0.0010
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.7318 - loss: 0.5200 - val_accuracy: 0.7647 - val_loss: 0.4827 - learning_rate: 0.0010
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.9490 - loss: 0.1540 - val_accuracy: 0.7647 - val_loss: 0.6800 - learning_rate: 0.0010
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.9894 - loss: 0.0421 - val_accuracy: 0.7706 - val_loss: 0.7431 - learning_rate: 0.0010
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9986 - loss: 0.0098
Epoch 7: 


Resultados LSTM:
  Loss: 1.1408
  Accuracy: 0.7925
  F1-Score: 0.7905
  ROC-AUC: 0.9010

Entrenando: GRU
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.



Resultados GRU:
  Loss: 0.6901
  Accuracy: 0.5472
  F1-Score: 0.7073
  ROC-AUC: 0.3917

Modelos principales entrenados: 4


In [14]:
# Entrenar modelos avanzados
if results and len(results) > 0:
    advanced_models = [
        (create_hybrid_model, "CNN + LSTM Hibrido"),
        (create_advanced_dnn, "DNN Avanzado")
    ]

    print("Entrenando modelos avanzados...")

    for model_func, model_name in advanced_models:
        try:
            result = train_and_evaluate_model(model_func, model_name, verbose=False)
            results.append(result)

        except Exception as e:
            print(f"Error entrenando {model_name}: {str(e)}")
            continue

    print(f"\nTotal modelos entrenados: {len(results)}")
else:
    print("No hay modelos base para entrenar avanzados")

Entrenando modelos avanzados...

Entrenando: CNN + LSTM Hibrido

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 9: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 5.

Resultados CNN + LSTM Hibrido:
  Loss: 0.8702
  Accuracy: 0.7877
  F1-Score: 0.8000
  ROC-AUC: 0.8456

Entrenando: DNN Avanzado

Epoch 11: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 6.

Resultados DNN Avanzado:
  Loss: 0.6494
  Accuracy: 0.7170
  F1-Score: 0.7059
  ROC-AUC: 0.8366

Total modelos entrenados: 6


## Analisis de Resultados

Comparo rendimiento de todas las arquitecturas entrenadas.

In [15]:
# Crear DataFrame resultados
if results:
    results_df = pd.DataFrame([
        {
            'Modelo': result['model_name'],
            'Test Accuracy': result['test_accuracy'],
            'F1-Score': result['f1_score'],
            'ROC-AUC': result['roc_auc'],
            'Test Loss': result['test_loss']
        }
        for result in results
    ])

    # Ordenar por F1-Score
    results_df = results_df.sort_values('F1-Score', ascending=False)

    print("Comparacion resultados arquitecturas DNN:")
    print(results_df.round(4))

    # Mejor modelo
    best_model_info = results_df.iloc[0]
    print(f"\nMejor modelo: {best_model_info['Modelo']}")
    print(f"F1-Score: {best_model_info['F1-Score']:.4f}")
    print(f"ROC-AUC: {best_model_info['ROC-AUC']:.4f}")
    print(f"Accuracy: {best_model_info['Test Accuracy']:.4f}")
else:
    print("No hay resultados para analizar")
    results_df = pd.DataFrame()

Comparacion resultados arquitecturas DNN:
               Modelo  Test Accuracy  F1-Score  ROC-AUC  Test Loss
4  CNN + LSTM Hibrido         0.7877    0.8000   0.8456     0.8702
2                LSTM         0.7925    0.7905   0.9010     1.1408
0          DNN Basico         0.7736    0.7714   0.8852     0.7545
1              CNN 1D         0.7500    0.7415   0.8035     1.0299
3                 GRU         0.5472    0.7073   0.3917     0.6901
5        DNN Avanzado         0.7170    0.7059   0.8366     0.6494

Mejor modelo: CNN + LSTM Hibrido
F1-Score: 0.8000
ROC-AUC: 0.8456
Accuracy: 0.7877


In [16]:
# Visualizar comparacion modelos
if not results_df.empty:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['F1-Score', 'ROC-AUC', 'Test Accuracy', 'Test Loss']
    )

    metrics = ['F1-Score', 'ROC-AUC', 'Test Accuracy', 'Test Loss']
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

    for metric, pos in zip(metrics, positions):
        color = 'lightblue' if metric != 'Test Loss' else 'lightcoral'
        fig.add_trace(
            go.Bar(
                x=results_df['Modelo'],
                y=results_df[metric],
                name=metric,
                showlegend=False,
                marker_color=color
            ),
            row=pos[0], col=pos[1]
        )

    fig.update_layout(
        title="Comparacion Rendimiento - Redes Neuronales Profundas",
        height=600
    )
    fig.update_xaxes(tickangle=45)
    fig.show()
else:
    print("No hay datos para visualizar")

## Curvas de Entrenamiento

Analizo curvas de entrenamiento para entender comportamiento.

In [17]:
# Visualizar curvas entrenamiento mejores modelos
if results:
    num_models = min(4, len(results))
    top_results = sorted(results, key=lambda x: x['f1_score'], reverse=True)[:num_models]

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[f"Loss - {result['model_name']}" for result in top_results[:2]] +
                       [f"Accuracy - {result['model_name']}" for result in top_results[:2]]
    )

    # Graficos loss y accuracy
    for i, result in enumerate(top_results[:2]):
        history = result['history']
        epochs = range(1, len(history.history['loss']) + 1)

        col = i + 1

        # Loss
        fig.add_trace(
            go.Scatter(x=list(epochs), y=history.history['loss'],
                      name='Train Loss', line=dict(color='blue')),
            row=1, col=col
        )
        fig.add_trace(
            go.Scatter(x=list(epochs), y=history.history['val_loss'],
                      name='Val Loss', line=dict(color='red')),
            row=1, col=col
        )

        # Accuracy
        fig.add_trace(
            go.Scatter(x=list(epochs), y=history.history['accuracy'],
                      name='Train Acc', line=dict(color='green')),
            row=2, col=col
        )
        fig.add_trace(
            go.Scatter(x=list(epochs), y=history.history['val_accuracy'],
                      name='Val Acc', line=dict(color='orange')),
            row=2, col=col
        )

    fig.update_layout(
        title="Curvas Entrenamiento - Mejores Modelos",
        height=600,
        showlegend=True
    )
    fig.show()
else:
    print("No hay resultados para curvas de entrenamiento")

In [18]:
# Analizar overfitting
if results:
    print("Analisis overfitting:")
    print("=" * 60)

    for result in results:
        history = result['history']
        model_name = result['model_name']

        # Diferencias train vs validacion
        final_train_loss = history.history['loss'][-1]
        final_val_loss = history.history['val_loss'][-1]
        final_train_acc = history.history['accuracy'][-1]
        final_val_acc = history.history['val_accuracy'][-1]

        loss_diff = final_val_loss - final_train_loss
        acc_diff = final_train_acc - final_val_acc

        print(f"\n{model_name}:")
        print(f"  Loss - Train: {final_train_loss:.4f}, Val: {final_val_loss:.4f}, Diff: {loss_diff:.4f}")
        print(f"  Acc - Train: {final_train_acc:.4f}, Val: {final_val_acc:.4f}, Diff: {acc_diff:.4f}")

        # Interpretar overfitting
        if loss_diff > 0.1 or acc_diff > 0.05:
            status = "POSIBLE OVERFITTING"
        elif loss_diff > 0.05 or acc_diff > 0.02:
            status = "Ligero overfitting"
        else:
            status = "Buen ajuste"

        print(f"  Estado: {status}")

    print("\n" + "=" * 60)
else:
    print("No hay datos para analisis overfitting")

Analisis overfitting:

DNN Basico:
  Loss - Train: 0.0007, Val: 0.7873, Diff: 0.7866
  Acc - Train: 1.0000, Val: 0.7706, Diff: 0.2294
  Estado: POSIBLE OVERFITTING

CNN 1D:
  Loss - Train: 0.0005, Val: 1.3941, Diff: 1.3936
  Acc - Train: 1.0000, Val: 0.7471, Diff: 0.2529
  Estado: POSIBLE OVERFITTING

LSTM:
  Loss - Train: 0.0125, Val: 1.0527, Diff: 1.0402
  Acc - Train: 0.9970, Val: 0.8353, Diff: 0.1617
  Estado: POSIBLE OVERFITTING

GRU:
  Loss - Train: 0.6897, Val: 0.6906, Diff: 0.0009
  Acc - Train: 0.5562, Val: 0.5294, Diff: 0.0268
  Estado: Ligero overfitting

CNN + LSTM Hibrido:
  Loss - Train: 0.0004, Val: 1.1005, Diff: 1.1001
  Acc - Train: 1.0000, Val: 0.7647, Diff: 0.2353
  Estado: POSIBLE OVERFITTING

DNN Avanzado:
  Loss - Train: 0.0411, Val: 0.6369, Diff: 0.5958
  Acc - Train: 0.9852, Val: 0.6294, Diff: 0.3558
  Estado: POSIBLE OVERFITTING



## Matrices de Confusion

Analizo comportamiento detallado con matrices de confusion.

In [19]:
# Matrices de confusion mejores modelos
if results:
    num_models_show = min(4, len(results))
    top_models = sorted(results, key=lambda x: x['f1_score'], reverse=True)[:num_models_show]

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[result['model_name'] for result in top_models],
        specs=[[{"type": "heatmap"} for _ in range(2)],
               [{"type": "heatmap"} for _ in range(2)]]
    )

    labels = ['Fake', 'Real']
    positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

    for i, (result, pos) in enumerate(zip(top_models, positions)):
        cm = confusion_matrix(y_test, result['y_pred'])

        # Normalizar
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        fig.add_trace(
            go.Heatmap(
                z=cm_normalized,
                x=labels,
                y=labels,
                colorscale='Blues',
                showscale=i == 0,
                text=[[f'{cm[i,j]}<br>({cm_normalized[i,j]:.2f})'
                       for j in range(len(labels))]
                      for i in range(len(labels))],
                texttemplate="%{text}",
                textfont={"size": 12}
            ),
            row=pos[0], col=pos[1]
        )

    fig.update_layout(
        title="Matrices Confusion - Redes Neuronales Profundas",
        height=600
    )
    fig.show()
else:
    print("No hay resultados para matrices de confusion")

In [20]:
# Reportes detallados clasificacion
if results:
    print("Reportes detallados clasificacion:")
    print("=" * 80)

    label_names = ['fake', 'real']
    top_models = sorted(results, key=lambda x: x['f1_score'], reverse=True)

    for result in top_models:
        print(f"\n{result['model_name'].upper()}:")
        print("-" * 50)

        # Reporte clasificacion
        report = classification_report(y_test, result['y_pred'],
                                     target_names=label_names,
                                     digits=4)
        print(report)

        # Metricas adicionales
        cm = confusion_matrix(y_test, result['y_pred'])
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn)  # Recall clase positiva
        specificity = tn / (tn + fp)  # Recall clase negativa

        print(f"Metricas adicionales:")
        print(f"  Sensibilidad (Recall Real): {sensitivity:.4f}")
        print(f"  Especificidad (Recall Fake): {specificity:.4f}")
        print(f"  TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

        print("-" * 50)
else:
    print("No hay resultados para reportes")

Reportes detallados clasificacion:

CNN + LSTM HIBRIDO:
--------------------------------------------------
              precision    recall  f1-score   support

        fake     0.7476    0.8021    0.7739        96
        real     0.8257    0.7759    0.8000       116

    accuracy                         0.7877       212
   macro avg     0.7866    0.7890    0.7869       212
weighted avg     0.7903    0.7877    0.7882       212

Metricas adicionales:
  Sensibilidad (Recall Real): 0.7759
  Especificidad (Recall Fake): 0.8021
  TP: 90, TN: 77, FP: 19, FN: 26
--------------------------------------------------

LSTM:
--------------------------------------------------
              precision    recall  f1-score   support

        fake     0.7203    0.8854    0.7944        96
        real     0.8830    0.7155    0.7905       116

    accuracy                         0.7925       212
   macro avg     0.8017    0.8005    0.7924       212
weighted avg     0.8093    0.7925    0.7922       212



## Comparacion con Modelos Tradicionales

Comparo DNN vs modelos tradicionales para evaluar mejora.

In [21]:
# Entrenar modelos tradicionales para comparacion
if 'X_train' in locals() and 'y_train' in locals():
    print("Entrenando modelos tradicionales...")

    # TF-IDF para modelos tradicionales
    tfidf_vectorizer = TfidfVectorizer(
        max_features=5000,
        stop_words='english',
        ngram_range=(1, 2)
    )

    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # Modelos tradicionales
    traditional_models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'SVM': SVC(probability=True, random_state=42),
        'Naive Bayes': MultinomialNB()
    }

    traditional_results = []

    for name, model in traditional_models.items():
        print(f"Entrenando {name}...")

        # Entrenar
        model.fit(X_train_tfidf, y_train)

        # Predicciones
        y_pred = model.predict(X_test_tfidf)
        y_pred_proba = model.predict_proba(X_test_tfidf)[:, 1]

        # Metricas
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)

        traditional_results.append({
            'Modelo': name,
            'Test Accuracy': accuracy,
            'F1-Score': f1,
            'ROC-AUC': auc,
            'Tipo': 'Tradicional'
        })

        print(f"  Acc: {accuracy:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    print("Modelos tradicionales entrenados")
else:
    print("Datos no disponibles para modelos tradicionales")
    traditional_results = []

Entrenando modelos tradicionales...
Entrenando Logistic Regression...
  Acc: 0.7877, F1: 0.8163, AUC: 0.8951
Entrenando Random Forest...
  Acc: 0.7500, F1: 0.7837, AUC: 0.8223
Entrenando SVM...
  Acc: 0.7972, F1: 0.8273, AUC: 0.8948
Entrenando Naive Bayes...
  Acc: 0.8208, F1: 0.8417, AUC: 0.9013
Modelos tradicionales entrenados


In [22]:
# Combinar resultados para comparacion
if results and traditional_results:
    dnn_results_comparison = [{
        'Modelo': result['model_name'],
        'Test Accuracy': result['test_accuracy'],
        'F1-Score': result['f1_score'],
        'ROC-AUC': result['roc_auc'],
        'Tipo': 'Deep Learning'
    } for result in results]

    # Combinar todos
    all_results = traditional_results + dnn_results_comparison
    comparison_df = pd.DataFrame(all_results)

    # Ordenar por F1-Score
    comparison_df = comparison_df.sort_values('F1-Score', ascending=False)

    print("Comparacion completa: Tradicionales vs DNN")
    print("=" * 80)
    print(comparison_df.round(4))

    # Mejores de cada tipo
    best_traditional = comparison_df[comparison_df['Tipo'] == 'Tradicional'].iloc[0]
    best_dnn = comparison_df[comparison_df['Tipo'] == 'Deep Learning'].iloc[0]

    print(f"\nMejor tradicional: {best_traditional['Modelo']} (F1: {best_traditional['F1-Score']:.4f})")
    print(f"Mejor DNN: {best_dnn['Modelo']} (F1: {best_dnn['F1-Score']:.4f})")

    improvement = ((best_dnn['F1-Score'] - best_traditional['F1-Score']) / best_traditional['F1-Score']) * 100
    print(f"Mejora DNN vs Tradicional: {improvement:.2f}%")
else:
    print("No hay suficientes resultados para comparacion")
    comparison_df = pd.DataFrame()
    improvement = 0

Comparacion completa: Tradicionales vs DNN
                Modelo  Test Accuracy  F1-Score  ROC-AUC           Tipo
3          Naive Bayes         0.8208    0.8417   0.9013    Tradicional
2                  SVM         0.7972    0.8273   0.8948    Tradicional
0  Logistic Regression         0.7877    0.8163   0.8951    Tradicional
8   CNN + LSTM Hibrido         0.7877    0.8000   0.8456  Deep Learning
6                 LSTM         0.7925    0.7905   0.9010  Deep Learning
1        Random Forest         0.7500    0.7837   0.8223    Tradicional
4           DNN Basico         0.7736    0.7714   0.8852  Deep Learning
5               CNN 1D         0.7500    0.7415   0.8035  Deep Learning
7                  GRU         0.5472    0.7073   0.3917  Deep Learning
9         DNN Avanzado         0.7170    0.7059   0.8366  Deep Learning

Mejor tradicional: Naive Bayes (F1: 0.8417)
Mejor DNN: CNN + LSTM Hibrido (F1: 0.8000)
Mejora DNN vs Tradicional: -4.95%


In [23]:
# Visualizar comparacion completa
if not comparison_df.empty:
    fig = px.scatter(
        comparison_df,
        x='F1-Score',
        y='ROC-AUC',
        color='Tipo',
        size='Test Accuracy',
        hover_data=['Modelo'],
        title="Comparacion: Tradicionales vs Redes Neuronales Profundas",
        color_discrete_map={'Tradicional': 'red', 'Deep Learning': 'blue'}
    )

    # Anotaciones mejores modelos
    if 'best_traditional' in locals():
        fig.add_annotation(
            x=best_traditional['F1-Score'],
            y=best_traditional['ROC-AUC'],
            text=f"Mejor Tradicional<br>{best_traditional['Modelo']}",
            showarrow=True,
            arrowhead=2
        )

    if 'best_dnn' in locals():
        fig.add_annotation(
            x=best_dnn['F1-Score'],
            y=best_dnn['ROC-AUC'],
            text=f"Mejor DNN<br>{best_dnn['Modelo']}",
            showarrow=True,
            arrowhead=2
        )

    fig.show()
    print("Grafico muestra posicion relativa modelos en F1-Score vs ROC-AUC")
    print("Tamano puntos representa Test Accuracy")
else:
    print("No hay datos para visualizar comparacion")

Grafico muestra posicion relativa modelos en F1-Score vs ROC-AUC
Tamano puntos representa Test Accuracy


## Resumen Final

Resumo hallazgos principales sobre DNN para deteccion desinformacion.

In [24]:
# Resumen final completo
print("=" * 90)
print("RESUMEN FINAL - REDES NEURONALES PROFUNDAS DETECCION DESINFORMACION")
print("=" * 90)

if results:
    print("\n1. ARQUITECTURAS IMPLEMENTADAS:")
    for i, result in enumerate(results, 1):
        print(f"   {i}. {result['model_name']}")

if 'VOCAB_SIZE' in locals():
    print("\n2. CONFIGURACION ENTRENAMIENTO:")
    print(f"   - Vocabulario: {VOCAB_SIZE:,}")
    print(f"   - Embedding dim: {EMBEDDING_DIM}")
    print(f"   - Longitud max secuencia: {MAX_SEQUENCE_LENGTH}")
    print(f"   - Epocas: {EPOCHS}")
    print(f"   - Batch size: {BATCH_SIZE}")
    if 'X_train' in locals():
        print(f"   - Datos entrenamiento: {len(X_train):,}")
        print(f"   - Datos prueba: {len(X_test):,}")

if results:
    print("\n3. RENDIMIENTO REDES NEURONALES:")
    dnn_sorted = sorted(results, key=lambda x: x['f1_score'], reverse=True)
    for i, result in enumerate(dnn_sorted, 1):
        print(f"   {i}. {result['model_name']:<20} F1: {result['f1_score']:.4f}, AUC: {result['roc_auc']:.4f}, Acc: {result['test_accuracy']:.4f}")

if 'best_traditional' in locals() and 'best_dnn' in locals():
    print("\n4. COMPARACION CON TRADICIONALES:")
    print(f"   Mejor tradicional: {best_traditional['Modelo']} (F1: {best_traditional['F1-Score']:.4f})")
    print(f"   Mejor DNN: {best_dnn['Modelo']} (F1: {best_dnn['F1-Score']:.4f})")
    print(f"   Mejora relativa: {improvement:.2f}%")

if results:
    print("\n5. ANALISIS OVERFITTING:")
    overfitting_count = 0
    for result in results:
        history = result['history']
        final_train_loss = history.history['loss'][-1]
        final_val_loss = history.history['val_loss'][-1]
        final_train_acc = history.history['accuracy'][-1]
        final_val_acc = history.history['val_accuracy'][-1]

        loss_diff = final_val_loss - final_train_loss
        acc_diff = final_train_acc - final_val_acc

        if loss_diff > 0.1 or acc_diff > 0.05:
            overfitting_count += 1

    print(f"   Modelos con posible overfitting: {overfitting_count}/{len(results)}")
    print(f"   Tecnicas regularizacion: Dropout, Early Stopping, ReduceLROnPlateau")

print("\n6. VENTAJAS REDES NEURONALES PROFUNDAS:")
print("   - Captura patrones complejos en texto")
print("   - Aprendizaje automatico representaciones")
print("   - Manejo eficiente secuencias largas (LSTM/GRU)")
print("   - Deteccion patrones locales (CNN)")
print("   - Arquitecturas hibridas combinan fortalezas")

print("\n7. LIMITACIONES IDENTIFICADAS:")
print("   - Requerimientos computacionales elevados")
print("   - Necesidad grandes cantidades datos")
print("   - Tendencia overfitting sin regularizacion")
print("   - Menor interpretabilidad vs tradicionales")
print("   - Sensibilidad hiperparametros")

print("\n8. RECOMENDACIONES:")
if results:
    best_result = max(results, key=lambda x: x['f1_score'])
    print(f"   - Usar {best_result['model_name']} como modelo principal")
print("   - Implementar ensemble multiples arquitecturas")
print("   - Considerar transfer learning modelos pre-entrenados")
print("   - Aplicar data augmentation mejorar generalizacion")
print("   - Monitorear metricas validacion durante entrenamiento")

print("\n" + "=" * 90)
print("Implementacion exitosa multiples arquitecturas DNN")
print("Mejoras significativas sobre enfoques tradicionales")
print("=" * 90)

RESUMEN FINAL - REDES NEURONALES PROFUNDAS DETECCION DESINFORMACION

1. ARQUITECTURAS IMPLEMENTADAS:
   1. DNN Basico
   2. CNN 1D
   3. LSTM
   4. GRU
   5. CNN + LSTM Hibrido
   6. DNN Avanzado

2. CONFIGURACION ENTRENAMIENTO:
   - Vocabulario: 3,407
   - Embedding dim: 128
   - Longitud max secuencia: 28
   - Epocas: 20
   - Batch size: 32
   - Datos entrenamiento: 846
   - Datos prueba: 212

3. RENDIMIENTO REDES NEURONALES:
   1. CNN + LSTM Hibrido   F1: 0.8000, AUC: 0.8456, Acc: 0.7877
   2. LSTM                 F1: 0.7905, AUC: 0.9010, Acc: 0.7925
   3. DNN Basico           F1: 0.7714, AUC: 0.8852, Acc: 0.7736
   4. CNN 1D               F1: 0.7415, AUC: 0.8035, Acc: 0.7500
   5. GRU                  F1: 0.7073, AUC: 0.3917, Acc: 0.5472
   6. DNN Avanzado         F1: 0.7059, AUC: 0.8366, Acc: 0.7170

4. COMPARACION CON TRADICIONALES:
   Mejor tradicional: Naive Bayes (F1: 0.8417)
   Mejor DNN: CNN + LSTM Hibrido (F1: 0.8000)
   Mejora relativa: -4.95%

5. ANALISIS OVERFITTING:
   

In [25]:
# Guardar resultados y modelos
import os
import pickle

# Directorio guardar
if IN_COLAB:
    save_dir = 'dnn_models'
else:
    save_dir = '../models/dnn_models'

os.makedirs(save_dir, exist_ok=True)

if not comparison_df.empty:
    # Guardar comparacion
    comparison_df.to_csv(f'{save_dir}/dnn_comparison_results.csv', index=False)
    print(f"Comparacion guardada: {save_dir}/dnn_comparison_results.csv")

if 'tokenizer' in locals():
    # Configuracion entrenamiento
    training_config = {
        'vocab_size': VOCAB_SIZE,
        'embedding_dim': EMBEDDING_DIM,
        'max_sequence_length': MAX_SEQUENCE_LENGTH,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'validation_split': VALIDATION_SPLIT
    }

    with open(f'{save_dir}/training_config.pickle', 'wb') as f:
        pickle.dump(training_config, f)

    # Tokenizer
    with open(f'{save_dir}/tokenizer.pickle', 'wb') as f:
        pickle.dump(tokenizer, f)

    print(f"Configuracion guardada: {save_dir}/training_config.pickle")
    print(f"Tokenizer guardado: {save_dir}/tokenizer.pickle")

if results:
    # Mejor modelo
    best_model = max(results, key=lambda x: x['f1_score'])['model']
    best_model.save(f'{save_dir}/best_dnn_model.h5')

    # Resultados detallados
    detailed_results = {
        'model_results': [
            {
                'model_name': result['model_name'],
                'test_accuracy': result['test_accuracy'],
                'f1_score': result['f1_score'],
                'roc_auc': result['roc_auc'],
                'test_loss': result['test_loss']
            }
            for result in results
        ],
        'best_model_name': max(results, key=lambda x: x['f1_score'])['model_name'],
        'improvement_over_traditional': improvement if 'improvement' in locals() else 0
    }

    with open(f'{save_dir}/detailed_results.pickle', 'wb') as f:
        pickle.dump(detailed_results, f)

    print(f"Mejor modelo guardado: {save_dir}/best_dnn_model.h5")
    print(f"Resultados detallados: {save_dir}/detailed_results.pickle")

print("\nArchivos generados:")
print("- dnn_comparison_results.csv: Comparacion completa modelos")
print("- training_config.pickle: Configuracion entrenamiento")
print("- tokenizer.pickle: Tokenizer entrenado")
print("- best_dnn_model.h5: Mejor modelo DNN")
print("- detailed_results.pickle: Resultados detallados")

print("\nImplementacion DNN completada exitosamente")

Comparacion guardada: dnn_models/dnn_comparison_results.csv
Configuracion guardada: dnn_models/training_config.pickle
Tokenizer guardado: dnn_models/tokenizer.pickle
Mejor modelo guardado: dnn_models/best_dnn_model.h5
Resultados detallados: dnn_models/detailed_results.pickle

Archivos generados:
- dnn_comparison_results.csv: Comparacion completa modelos
- training_config.pickle: Configuracion entrenamiento
- tokenizer.pickle: Tokenizer entrenado
- best_dnn_model.h5: Mejor modelo DNN
- detailed_results.pickle: Resultados detallados

Implementacion DNN completada exitosamente
